# GA Data Science 16 (DAT16) - Lab 17 a

### Time Series Analysis

Francesco Mosconi, Justin Breucop

### Today

In [ ]:
%pylab inline
import pandas as pd

## Sound Analysis

In [ ]:
from scipy.io import wavfile
fs, data = wavfile.read('./Time Series Audio.wav')

In [ ]:
mono = data[:,1]
T = len(mono)/44100.0
t = np.linspace(0,T, len(mono))

display the spectrogram

In [ ]:
figure(figsize = (15,10))
subplot(211)
plot(t,mono)
title("sound")

subplot(212)
# the specgram function already outputs a plot
_ = specgram(mono, Fs = 44100, NFFT=512)
ylim(0,15000)
title("Spectrogram")
ylabel("Frequency (Hz)")

In [ ]:
figure(figsize = (15,5))
# change the resolution of the specgram
_ = specgram(mono, Fs = 44100, NFFT=1024)
ylim(0,15000)
title("Spectrogram")
ylabel("Frequency (Hz)")

In [ ]:
figure(figsize = (15,5))
_ = specgram(mono, Fs = 44100, NFFT=2048)
ylim(0,15000)
title("Spectrogram")
ylabel("Frequency (Hz)")

In [ ]:
figure(figsize = (10,7))
ts = pd.Series(randn(4000), index=pd.date_range('1/1/2000', periods=4000))
ts = ts.cumsum()
ts.plot()

## fancy plotting

In [ ]:
from pandas.tools.plotting import lag_plot

In [ ]:
corrts = pd.Series(0.2 * rand(1000) + 0.8 * np.sin(np.linspace(-99 * np.pi, 99 * np.pi, num=1000)))

In [ ]:
figure(figsize = (15,10))
subplot(221)
ts.plot()
subplot(222)
lag_plot(ts, alpha = 0.5)
subplot(223)
corrts.ix[0:200].plot()

subplot(224)
lag_plot(corrts, alpha = 0.5)


In [ ]:
from pandas.tools.plotting import autocorrelation_plot

In [ ]:
figure(figsize = (15,10))
subplot(221)
ts.plot()
ax = subplot(222)
autocorrelation_plot(ts, alpha = 0.5, ax = ax)
subplot(223)
corrts.ix[0:200].plot()

ax1= subplot(224)
autocorrelation_plot(corrts, alpha = 0.5, ax = ax1)


In [ ]:
from bokeh.sampledata.stocks import MSFT
import bokeh.plotting as bh
bh.output_notebook()

In [ ]:
df = pd.DataFrame(MSFT)[:50]
df["date"] = pd.to_datetime(df["date"])

mids = (df.open + df.close)/2
spans = abs(df.close-df.open)

inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1000 # half day in ms


p = bh.figure(x_axis_type="datetime", tools='', plot_width=1000)

p.segment(df.date, df.high, df.date, df.low, color="black", toolbar_location="left")
p.rect(df.date[inc], mids[inc], w, spans[inc], fill_color="#D5E1DD", line_color="black")
p.rect(df.date[dec], mids[dec], w, spans[dec], fill_color="#F2583E", line_color="black")

p.title = "MSFT Candlestick"
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3
bh.show(p)

In [ ]:
figure(figsize = (15,10))
subplot(221)
ts.plot()

subplot(222)
ts.plot(alpha = 0.3)

pd.rolling_mean(ts, 20).plot(alpha = 1.0)
legend(['data','rolling mean 20 points'])

subplot(223)
ts.plot(alpha = 0.3)

pd.rolling_mean(ts, 200).plot(alpha = 1.0)
legend(['data','rolling mean 200 points'])

subplot(224)
ts.plot(alpha = 0.3)

pd.rolling_mean(ts, 500).plot(alpha = 1.0)
legend(['data','rolling mean 500 points'])


In [ ]:
from scipy.signal import butter, filtfilt

figure(figsize = (15,10))
subplot(221)
ts.plot()

subplot(222)
ts.plot(alpha = 0.3)

pd.rolling_mean(ts, 200, center = True).plot(alpha = 1.0)
legend(['data','rolling mean 200 points'])

subplot(223)
ts.plot(alpha = 0.3)

pd.stats.moments.ewma(ts, 160).plot(alpha = 1.0)
legend(['data','EWMA 160 points'])

subplot(224)
ts.plot(alpha = 0.3)

pd.Series(data = filtfilt(*butter(4,0.008,'low'), x = ts.values), index = ts.index).plot(alpha = 1.0)
legend(['data','Butterworth 4th order IIR 0.008'])


In [ ]:
figure(figsize = (15,10))
subplot(211)
(ts+50).plot()
axhline(50, color = 'r')
axhline(50+ts.std())
axhline(50-ts.std())

subplot(212)
(ts/ts.std()).plot()
axhline(0, color = 'r')
axhline(1)
axhline(-1)

In [ ]:
ttt = np.linspace(0, 10, num=1000)
noisysine = pd.Series(0.5 * rand(1000) + 0.5 * np.sin(2*pi*ttt))

figure(figsize = (15,10))
subplot(221)
noisysine.plot()

subplot(222)
(noisysine.diff()*2*pi).plot()
title('difference')

subplot(223)
noisysine.plot(alpha = 0.3)
filtered = pd.Series(data = filtfilt(*butter(4,0.05,'low'), x = noisysine.values), index = noisysine.index)
filtered.plot(alpha = 1.0)
legend(['data','Butterworth 4th order IIR 0.05'])

subplot(224)
(noisysine.diff()*2*pi).plot(alpha = 0.3)
(filtered.diff()*2*pi).plot(alpha = 1.0)
filtered.plot(alpha = 0.5)

legend(['diff of raw', 'diff of filtered', 'filtered'])

In [ ]:
t = linspace(0,10,1000)
noise = 0.3*randn(1000)
periodic = sin(5*t)
trend = 0.5*t
amplif = exp(0.1*t)


plot(t, (trend+periodic+noise)*amplif)
title('Timeseries')
xlabel('Time')
xlim(0,12)


In [ ]:
plot(t, (trend+periodic+noise)*amplif)
plot(t, trend*amplif)
plot(t, (trend+periodic)*amplif)

title('Timeseries')
xlabel('Time')
xlim(0,12)


In [ ]:
subplot(411)
plot(t, noise)
xlim(0,12)
title('Noise')

subplot(412)
plot(t, trend)
xlim(0,12)
title('Trend')

subplot(413)
plot(t, periodic)
title('Periodic')
xlim(0,12)


subplot(414)
plot(t, amplif)
title('Gain')

xlabel('Time')
xlim(0,12)

In [ ]:
airpassdata = pd.read_csv('lab_data/international-airline-passengers.csv')
airpassdata['Month'] = pd.to_datetime(airpassdata['Month'])
airpassdata.set_index('Month', inplace=True)
airpassdata.head()

In [ ]:
airpassdata.plot()